In [2]:
!pip install dash

     |████████████████████████████████| 7.3 MB 17.8 MB/s eta 0:00:01
     |████████████████████████████████| 23.9 MB 83.6 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 6.7 MB/s s eta 0:00:01
     |████████████████████████████████| 288 kB 87.3 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 90.3 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=83e3e75613730ac2d68e4e4831dfd1160ca23febd8865a73dbcc9ca4cad5ff80
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=114d95c62fb0f9a31226d12930435f17c43faedb2f19232aa5f4e5c1dae78b20
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                  dcc.Dropdown(id='site_dropdown',
                                                options=[
                                                   {'label': 'All Sites', 'value': 'ALL'},
                                                   {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                   {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                   {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                   {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                ],
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       3000: '3000',
                                                       6000: '6000',
                                                       10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site_dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        s=spacex_df.groupby('Launch Site').mean()
        fig = px.pie(s, values='class',names=s.index)
    else:
        # return the outcomes piechart for a selected site
        s=spacex_df[spacex_df['Launch Site']==entered_site]
        s=s.groupby('Launch Site').mean()
        dict={'cat':['Success','Failure'], 'Value':[s['class'][0],1-s['class'][0]]}
        s=pd.DataFrame(dict)
        fig = px.pie(s, values='Value',names='cat')
    return fig       


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site_dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, slide):
    val1=slide[0]
    val2=slide[0]
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>slide[0]) & (spacex_df['Payload Mass (kg)']<slide[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)',y='class',color="Booster Version Category")
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df2 = filtered_df[filtered_df['Launch Site']==entered_site]
        fig = px.scatter(filtered_df2, x='Payload Mass (kg)',y='class',color="Booster Version Category")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


<ipython-input-3-4244c9525ce3>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-3-4244c9525ce3>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
